In [1]:
!curl -L -o cats-and-dogs-mini-dataset.zip\
  https://www.kaggle.com/api/v1/datasets/download/aleemaparakatta/cats-and-dogs-mini-dataset

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:01 --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:01 --:--:--     0

  0     0    0     0    0     0      0      0 --:--:--  0:00:02 --:--:--     0
  0 21.8M    0 50106    0     0  15756      0  0:24:15  0:00:03  0:24:12 52910
  2 21.8M    2  475k    0     0   116k      0  0:03:12  0:00:04  0:03:08  255k
  7 21.8M    7 1598k    0     0   313k      0  0:01:11  0:00:05  0:01:06  558k
  9 21.8M    9 2144k    0     0   350k      0  0:01:03  0:00:06  0:00:57  553k
 12 21.8M   12 2704k    0     0   381k      0  0:00:58  0:00:07  0:00:51  556k
 14 21.8M   14 3312k    0     0   408k      0  0:0

In [ ]:
### Creating Dataset Structure for Training and Inferencing

!mkdir data


'unzip' is not recognized as an internal or external command,
operable program or batch file.


In [6]:
import zipfile

# Path to the ZIP file
zip_path = "cats-and-dogs-mini-dataset.zip"
extract_path = "dataset"

# Open the ZIP file and extract
with zipfile.ZipFile(zip_path, "r") as zip_ref:
    zip_ref.extractall(extract_path)

print(f"Extracted files to {extract_path}")


Extracted files to dataset


In [7]:
import os

# Create the 'data' directory if it doesn't exist
data_dir = 'data'
os.makedirs(data_dir, exist_ok=True)

# Create the 'train' and 'test' subdirectories within 'data'
train_dir = os.path.join(data_dir, 'train')
test_dir = os.path.join(data_dir, 'test')
os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

In [11]:
import os

# Define the source and destination paths
source_cat = 'dataset/cats_set'
dest_cat = 'dataset/cat'
source_dog = 'dataset/dogs_set'
dest_dog = 'dataset/dog'

# Rename the directories if they exist
if os.path.exists(source_cat):
  os.rename(source_cat, dest_cat)

if os.path.exists(source_dog):
  os.rename(source_dog, dest_dog)

In [12]:
### Creating the TRAIN TEST SPLIT within the DATA directory folders
import os
import random
import shutil

def train_test_split_folder(source_folder, train_folder, test_folder, split_ratio=0.8):
    """
    Splits a folder of images into training and testing sets.

    Args:
        source_folder: Path to the source folder containing subfolders for each class.
        train_folder: Path to the folder where the training set will be saved.
        test_folder: Path to the folder where the testing set will be saved.
        split_ratio: The ratio of images to include in the training set (default is 0.8).
    """

    if not os.path.exists(train_folder):
        os.makedirs(train_folder)
    if not os.path.exists(test_folder):
        os.makedirs(test_folder)

    for class_name in os.listdir(source_folder):
        class_source_path = os.path.join(source_folder, class_name)

        if os.path.isdir(class_source_path):  # Check if it is a directory
            train_class_path = os.path.join(train_folder, class_name)
            test_class_path = os.path.join(test_folder, class_name)

            if not os.path.exists(train_class_path):
                os.makedirs(train_class_path)
            if not os.path.exists(test_class_path):
                os.makedirs(test_class_path)

            images = [f for f in os.listdir(class_source_path) if os.path.isfile(os.path.join(class_source_path, f))]
            random.shuffle(images)
            split_index = int(len(images) * split_ratio)
            train_images = images[:split_index]
            test_images = images[split_index:]

            for image in train_images:
                source_path = os.path.join(class_source_path, image)
                destination_path = os.path.join(train_class_path, image)
                shutil.copy(source_path, destination_path)

            for image in test_images:
                source_path = os.path.join(class_source_path, image)
                destination_path = os.path.join(test_class_path, image)
                shutil.copy(source_path, destination_path)


# Example usage (assuming you have your data organized in a 'data/train' folder):
train_test_split_folder("dataset", "data/train", "data/test")

In [21]:
import os

def count_images_per_folder(root_folder):
    """
    Counts the number of images in each subfolder of a given root folder.

    Args:
    root_folder: The path to the root folder.

    Returns:
    A dictionary where keys are folder paths and values are the number of images in each folder.
    """

    image_counts = {}
    for dirpath, dirnames, filenames in os.walk(root_folder):
        file = [f for f in filenames if f[-4:] in ['.jpg', '.jpeg', '.png', '.gif']]
        
        if len(file) > 0: 
            image_counts[dirpath] = len(file)
    return image_counts


image_counts = count_images_per_folder("data")

for folder, count in image_counts.items():
    print(f"Folder {folder} count{count}")

Folder data\test\cat count100
Folder data\test\dog count100
Folder data\train\cat count400
Folder data\train\dog count400


In [26]:
!tree -d data/train

Too many parameters - data/train


In [27]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1' 

import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint

In [ ]:
# Set paths to train and test folders
train_dir = "data/train"
test_dir = "data/test"
image_size = (128, 128)
batch_size = 32

In [ ]:
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)


train_data = datagen.flow_from_directory(
    train_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='binary',
    subset='training'
)

test_data = datagen.flow_from_directory(
    test_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='binary',
    subset='validation'
)

In [ ]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # binary classification
])
model.summary()

In [ ]:
model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

# Training the Model
checkpoint_path = 'dog_cat_cnn_model.keras'
checkpoint = ModelCheckpoint(checkpoint_path, monitor='val_accuracy', save_best_only=True)

model.fit(
    train_data,
    epochs=10,
    validation_data=test_data,
    callbacks=[checkpoint]
)

In [ ]:
# Save final model
model.save('dog_cat_final_model.keras')

In [ ]:
from tensorflow.keras.models import load_model

# Load the best saved model
model = load_model('dog_cat_final_model.keras')

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Set parameters
val_dir = test_dir  # same directory used for training
image_size = (128, 128)
batch_size = 32

# Recreate the validation generator
val_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.25)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='binary',
    subset='validation'
)

# Evaluate model accuracy
loss, accuracy = model.evaluate(val_generator)
print(f"Validation Accuracy: {accuracy * 100:.2f}%")

In [ ]:
from tensorflow.keras.preprocessing import image
import numpy as np

def predict_image(img_path, model):
    img = image.load_img(img_path, target_size=(128, 128))
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    prediction = model.predict(img_array)[0][0]

    if prediction > 0.5:
        print("Predicted: Dog")
    else:
        print("Predicted: Cat")